In [36]:
"""
Implementation of logistic ordinal regression (aka proportional odds) model
"""

from __future__ import print_function

from sklearn import metrics
from scipy import linalg, optimize, sparse
import numpy as np
import warnings

BIG = 1e10
SMALL = 1e-12


def phi(t):
    """
    logistic function, returns 1 / (1 + exp(-t))
    """
    idx = t > 0
    out = np.empty(t.size, dtype=np.float)
    out[idx] = 1. / (1 + np.exp(-t[idx]))
    exp_t = np.exp(t[~idx])
    out[~idx] = exp_t / (1. + exp_t)
    return out

def log_logistic(t):
    """
    (minus) logistic loss function, returns log(1 / (1 + exp(-t)))
    """
    idx = t > 0
    out = np.zeros_like(t)
    out[idx] = np.log(1 + np.exp(-t[idx]))
    out[~idx] = (-t[~idx] + np.log(1 + np.exp(t[~idx])))
    return out


def ordinal_logistic_fit(X, y, alpha=0, l1_ratio=0, n_class=None, max_iter=10000,
                         verbose=False, solver='TNC', w0=None):
    """
    Ordinal logistic regression or proportional odds model.
    Uses scipy's optimize.fmin_slsqp solver.
    Parameters
    ----------
    X : {array, sparse matrix}, shape (n_samples, n_feaures)
        Input data
    y : array-like
        Target values
    max_iter : int
        Maximum number of iterations
    verbose: bool
        Print convergence information
    Returns
    -------
    w : array, shape (n_features,)
        coefficients of the linear model
    theta : array, shape (k,), where k is the different values of y
        vector of thresholds
    """

    X = np.asarray(X)
    y = np.asarray(y)
    w0 = None

    if not X.shape[0] == y.shape[0]:
        raise ValueError('Wrong shape for X and y')

    # .. order input ..
    idx = np.argsort(y)
    idx_inv = np.zeros_like(idx)
    idx_inv[idx] = np.arange(idx.size)
    X = X[idx]
    y = y[idx].astype(np.int)
    # make them continuous and start at zero
    unique_y = np.unique(y)
    for i, u in enumerate(unique_y):
        y[y == u] = i
    unique_y = np.unique(y)

    # .. utility arrays used in f_grad ..
    alpha = 0.
    k1 = np.sum(y == unique_y[0])
    E0 = (y[:, np.newaxis] == np.unique(y)).astype(np.int)
    E1 = np.roll(E0, -1, axis=-1)
    E1[:, -1] = 0.
    E0, E1 = map(sparse.csr_matrix, (E0.T, E1.T))

    def f_obj(x0, X, y):
        """
        Objective function
        """
        w, theta_0 = np.split(x0, [X.shape[1]])
        theta_1 = np.roll(theta_0, 1)
        t0 = theta_0[y]
        z = np.diff(theta_0)

        Xw = X.dot(w)
        a = t0 - Xw
        b = t0[k1:] - X[k1:].dot(w)
        c = (theta_1 - theta_0)[y][k1:]

        if np.any(c > 0):
            return BIG

        #loss = -(c[idx] + np.log(np.exp(-c[idx]) - 1)).sum()
        loss = -np.log(1 - np.exp(c)).sum()

        loss += b.sum() + log_logistic(b).sum() \
            + log_logistic(a).sum() \
            + .5 * alpha * w.dot(w) - np.log(z).sum()  # penalty
        if np.isnan(loss):
            pass
            #import ipdb; ipdb.set_trace()
        return loss

    def f_grad(x0, X, y):
        """
        Gradient of the objective function
        """
        w, theta_0 = np.split(x0, [X.shape[1]])
        theta_1 = np.roll(theta_0, 1)
        t0 = theta_0[y]
        t1 = theta_1[y]
        z = np.diff(theta_0)

        Xw = X.dot(w)
        a = t0 - Xw
        b = t0[k1:] - X[k1:].dot(w)
        c = (theta_1 - theta_0)[y][k1:]

        # gradient for w
        phi_a = phi(a)
        phi_b = phi(b)
        grad_w = -X[k1:].T.dot(phi_b) + X.T.dot(1 - phi_a) + alpha * w

        # gradient for theta
        idx = c > 0
        tmp = np.empty_like(c)
        tmp[idx] = 1. / (np.exp(-c[idx]) - 1)
        tmp[~idx] = np.exp(c[~idx]) / (1 - np.exp(c[~idx])) # should not need
        grad_theta = (E1 - E0)[:, k1:].dot(tmp) \
            + E0[:, k1:].dot(phi_b) - E0.dot(1 - phi_a)

        grad_theta[:-1] += 1. / np.diff(theta_0)
        grad_theta[1:] -= 1. / np.diff(theta_0)
        out = np.concatenate((grad_w, grad_theta))
        return out

    def f_hess(x0, s, X, y):
        x0 = np.asarray(x0)
        w, theta_0 = np.split(x0, [X.shape[1]])
        theta_1 = np.roll(theta_0, 1)
        t0 = theta_0[y]
        t1 = theta_1[y]
        z = np.diff(theta_0)

        Xw = X.dot(w)
        a = t0 - Xw
        b = t0[k1:] - X[k1:].dot(w)
        c = (theta_1 - theta_0)[y][k1:]

        D = np.diag(phi(a) * (1 - phi(a)))
        D_= np.diag(phi(b) * (1 - phi(b)))
        D1 = np.diag(np.exp(-c) / (np.exp(-c) - 1) ** 2)
        Ex = (E1 - E0)[:, k1:].toarray()
        Ex0 = E0.toarray()
        H_A = X[k1:].T.dot(D_).dot(X[k1:]) + X.T.dot(D).dot(X)
        H_C = - X[k1:].T.dot(D_).dot(E0[:, k1:].T.toarray()) \
            - X.T.dot(D).dot(E0.T.toarray())
        H_B = Ex.dot(D1).dot(Ex.T) + Ex0[:, k1:].dot(D_).dot(Ex0[:, k1:].T) \
            - Ex0.dot(D).dot(Ex0.T)

        p_w = H_A.shape[0]
        tmp0 = H_A.dot(s[:p_w]) + H_C.dot(s[p_w:])
        tmp1 = H_C.T.dot(s[:p_w]) + H_B.dot(s[p_w:])
        return np.concatenate((tmp0, tmp1))

        import ipdb; ipdb.set_trace()
        import pylab as pl
        pl.matshow(H_B)
        pl.colorbar()
        pl.title('True')
        import numdifftools as nd
        Hess = nd.Hessian(lambda x: f_obj(x, X, y))
        H = Hess(x0)
        pl.matshow(H[H_A.shape[0]:, H_A.shape[0]:])
        #pl.matshow()
        pl.title('estimated')
        pl.colorbar()
        pl.show()


    def grad_hess(x0, X, y):
        grad = f_grad(x0, X, y)
        hess = lambda x: f_hess(x0, x, X, y)
        return grad, hess

    x0 = np.random.randn(X.shape[1] + unique_y.size) / X.shape[1]
    if w0 is not None:
        x0[:X.shape[1]] = w0
    else:
        x0[:X.shape[1]] = 0.
    x0[X.shape[1]:] = np.sort(unique_y.size * np.random.rand(unique_y.size))

    #print('Check grad: %s' % optimize.check_grad(f_obj, f_grad, x0, X, y))
    #print(optimize.approx_fprime(x0, f_obj, 1e-6, X, y))
    #print(f_grad(x0, X, y))
    #print(optimize.approx_fprime(x0, f_obj, 1e-6, X, y) - f_grad(x0, X, y))
    #import ipdb; ipdb.set_trace()

    def callback(x0):
        x0 = np.asarray(x0)
        # print('Check grad: %s' % optimize.check_grad(f_obj, f_grad, x0, X, y))
        if verbose:
        # check that gradient is correctly computed
            print('OBJ: %s' % f_obj(x0, X, y))

    if solver == 'TRON':
        import pytron
        out = pytron.minimize(f_obj, grad_hess, x0, args=(X, y))
    else:
        options = {'maxiter' : max_iter, 'disp': 0, 'maxfun':10000}
        out = optimize.minimize(f_obj, x0, args=(X, y), method=solver,
            jac=f_grad, hessp=f_hess, options=options, callback=callback)

    if not out.success:
        warnings.warn(out.message)
    w, theta = np.split(out.x, [X.shape[1]])
    return w, theta


def ordinal_logistic_predict(w, theta, X):
    """
    Parameters
    ----------
    w : coefficients obtained by ordinal_logistic
    theta : thresholds
    """
    unique_theta = np.sort(np.unique(theta))
    out = X.dot(w)
    unique_theta[-1] = np.inf # p(y <= max_level) = 1
    tmp = out[:, None].repeat(unique_theta.size, axis=1)
    return np.argmax(tmp < unique_theta, axis=1)



In [38]:
if __name__ == '__main__':
    DOC = """
================================================================================
    Compare the prediction accuracy of different models on the boston dataset
================================================================================
    """
    print(DOC)
    from sklearn import cross_validation, datasets
    boston = datasets.load_boston()
#     print(boston.data,boston.feature_names, boston.target)
    X, y = boston.data, np.round(boston.target)
   
    


    Compare the prediction accuracy of different models on the boston dataset
    


In [43]:
# X -= X.mean()
y -= y.min()

In [50]:
cv = cross_validation.ShuffleSplit(y.size, n_iter=50, test_size=0, random_state=0)
score_logistic = []
score_ordinal_logistic = []
score_ridge = []

In [51]:
print(cv)
print(test)

ShuffleSplit(506, n_iter=50, test_size=0, random_state=0)
[ 14  24  25  27  31  44  45  55  57  77  88 103 111 130 141 160 168 175
 183 203 227 255 256 271 275 279 287 289 291 296 299 308 311 314 319 326
 330 363 368 369 373 386 387 406 423 426 462 466 469 472 478]


In [32]:
for i, (train, test) in enumerate(cv):
    #test = train
    if not np.all(np.unique(y[train]) == np.unique(y)):
        # we need the train set to have all different classes
        continue
    assert np.all(np.unique(y[train]) == np.unique(y))
    train = np.sort(train)
    test = np.sort(test)
    w, theta = ordinal_logistic_fit(X[train], y[train], verbose=True,
                                    solver='TNC')
    pred = ordinal_logistic_predict(w, theta, X[test])
    1/0
    s = metrics.mean_absolute_error(y[test], pred)
    print('ERROR (ORDINAL)  fold %s: %s' % (i+1, s))
    score_ordinal_logistic.append(s)

    from sklearn import linear_model
    clf = linear_model.LogisticRegression(C=1.)
    clf.fit(X[train], y[train])
    pred = clf.predict(X[test])
    s = metrics.mean_absolute_error(y[test], pred)
    print('ERROR (LOGISTIC) fold %s: %s' % (i+1, s))
    score_logistic.append(s)

    from sklearn import linear_model
    clf = linear_model.Ridge(alpha=1.)
    clf.fit(X[train], y[train])
    pred = np.round(clf.predict(X[test]))
    s = metrics.mean_absolute_error(y[test], pred)
    print('ERROR (RIDGE)    fold %s: %s' % (i+1, s))
    score_ridge.append(s)

/Users/maryam/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/maryam/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:219: OptimizeWarning: Unknown solver options: maxfun


OBJ: 3836.39643989
OBJ: 3242.45506778
OBJ: 2888.61515037
OBJ: 2724.77110508
OBJ: 2661.27089819
OBJ: 2469.71354914
OBJ: 2318.18112918
OBJ: 2237.17077769
OBJ: 2184.72474691
OBJ: 1981.83573563
OBJ: 1924.187787
OBJ: 1878.99363192
OBJ: 1817.96615032
OBJ: 1812.14228582
OBJ: 1805.16100241
OBJ: 1665.3030419
OBJ: 1639.31976828
OBJ: 1626.38841872
OBJ: 1613.44829009
OBJ: 1592.85324032
OBJ: 1575.59259395
OBJ: 1560.55013888
OBJ: 1525.98550451
OBJ: 1509.81084072
OBJ: 1490.75931305
OBJ: 1464.54204676
OBJ: 1421.91921257
OBJ: 1394.78716069
OBJ: 1387.28477631
OBJ: 1372.06857887
OBJ: 1365.77496971
OBJ: 1359.89368138
OBJ: 1359.31691479
OBJ: 1359.02151926
OBJ: 1356.01272689
OBJ: 1355.5537549
OBJ: 1355.49568123
OBJ: 1352.24027894
OBJ: 1349.4168109
OBJ: 1348.8463026
OBJ: 1346.68566515
OBJ: 1346.57088012
OBJ: 1345.85195546
OBJ: 1345.70859976
OBJ: 1344.97692805
OBJ: 1344.88934875
OBJ: 1343.27988949
OBJ: 1341.50544428
OBJ: 1335.57424913
OBJ: 1334.68653214
OBJ: 1333.20707535
OBJ: 1332.92941811
OBJ: 1332.45371135

OBJ: 1323.8205302
OBJ: 1323.82036889
OBJ: 1323.81996586
OBJ: 1323.81984229
OBJ: 1323.81982114
OBJ: 1323.81919644
OBJ: 1323.81809386
OBJ: 1323.81621857
OBJ: 1323.81566799
OBJ: 1323.81548423
OBJ: 1323.81463822
OBJ: 1323.81417395
OBJ: 1323.8064239
OBJ: 1323.79852642
OBJ: 1323.79726517
OBJ: 1323.78892081
OBJ: 1323.78502837
OBJ: 1323.7846583
OBJ: 1323.78456481
OBJ: 1323.78230207
OBJ: 1323.78216071
OBJ: 1323.78191975
OBJ: 1323.78190143
OBJ: 1323.7816807
OBJ: 1323.78150956
OBJ: 1323.78122066
OBJ: 1323.78065098
OBJ: 1323.77978567
OBJ: 1323.77887045
OBJ: 1323.7775291
OBJ: 1323.77585735
OBJ: 1323.77538373
OBJ: 1323.77374621
OBJ: 1323.77341919
OBJ: 1323.77102369
OBJ: 1323.76471952
OBJ: 1323.76422559
OBJ: 1323.7641506
OBJ: 1323.76322127
OBJ: 1323.76310571
OBJ: 1323.76273785
OBJ: 1323.76254786
OBJ: 1323.76253237
OBJ: 1323.7622901
OBJ: 1323.76139327
OBJ: 1323.76127596
OBJ: 1323.76121512
OBJ: 1323.76090312
OBJ: 1323.76030869
OBJ: 1323.76023974
OBJ: 1323.75956975
OBJ: 1323.75794406
OBJ: 1323.75718829


/Users/maryam/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:222: UserWarning: Max. number of function evaluations reached


ZeroDivisionError: integer division or modulo by zero

In [29]:
X -= X.mean()
y -= y.min()

idx = np.argsort(y)
X = X[idx]
y = y[idx]
cv = cross_validation.ShuffleSplit(y.size, n_iter=50, test_size=.1, random_state=0)
score_logistic = []
score_ordinal_logistic = []
score_ridge = []
for i, (train, test) in enumerate(cv):
    #test = train
    if not np.all(np.unique(y[train]) == np.unique(y)):
        # we need the train set to have all different classes
        continue
    assert np.all(np.unique(y[train]) == np.unique(y))
    train = np.sort(train)
    test = np.sort(test)
    w, theta = ordinal_logistic_fit(X[train], y[train], verbose=True,
                                    solver='TNC')
    pred = ordinal_logistic_predict(w, theta, X[test])
    1/0
    s = metrics.mean_absolute_error(y[test], pred)
    print('ERROR (ORDINAL)  fold %s: %s' % (i+1, s))
    score_ordinal_logistic.append(s)

    from sklearn import linear_model
    clf = linear_model.LogisticRegression(C=1.)
    clf.fit(X[train], y[train])
    pred = clf.predict(X[test])
    s = metrics.mean_absolute_error(y[test], pred)
    print('ERROR (LOGISTIC) fold %s: %s' % (i+1, s))
    score_logistic.append(s)

    from sklearn import linear_model
    clf = linear_model.Ridge(alpha=1.)
    clf.fit(X[train], y[train])
    pred = np.round(clf.predict(X[test]))
    s = metrics.mean_absolute_error(y[test], pred)
    print('ERROR (RIDGE)    fold %s: %s' % (i+1, s))
    score_ridge.append(s)


print()
print('MEAN ABSOLUTE ERROR (ORDINAL LOGISTIC):    %s' % np.mean(score_ordinal_logistic))
print('MEAN ABSOLUTE ERROR (LOGISTIC REGRESSION): %s' % np.mean(score_logistic))
print('MEAN ABSOLUTE ERROR (RIDGE REGRESSION):    %s' % np.mean(score_ridge))
# print('Chance level is at %s' % (1. / np.unique(y).size))


/Users/maryam/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  del sys.path[0]
/Users/maryam/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:219: OptimizeWarning: Unknown solver options: maxfun


OBJ: 4393.47818448
OBJ: 3169.54741094
OBJ: 2865.91348681
OBJ: 2675.46758506
OBJ: 2419.49848579
OBJ: 2292.07167923
OBJ: 2090.46101133
OBJ: 2024.75507852
OBJ: 1971.3519521
OBJ: 1934.81872919
OBJ: 1916.04367528
OBJ: 1878.0824913
OBJ: 1850.95467247
OBJ: 1835.24928112
OBJ: 1793.52710795
OBJ: 1771.83390253
OBJ: 1698.34628659
OBJ: 1674.87511243
OBJ: 1662.88922312
OBJ: 1610.01719669
OBJ: 1588.10360517
OBJ: 1546.09939429
OBJ: 1534.38256332
OBJ: 1511.64420543
OBJ: 1480.60654913
OBJ: 1421.03520876
OBJ: 1407.25998994
OBJ: 1373.65097995
OBJ: 1369.61919213
OBJ: 1366.19775492
OBJ: 1356.74050864
OBJ: 1353.26677039
OBJ: 1352.8873939
OBJ: 1352.3779472
OBJ: 1352.03780058
OBJ: 1348.92257291
OBJ: 1345.07326649
OBJ: 1343.05991139
OBJ: 1342.62394393
OBJ: 1341.50507081
OBJ: 1341.46759461
OBJ: 1341.45509326
OBJ: 1341.22629242
OBJ: 1341.04062816
OBJ: 1340.54086932
OBJ: 1340.1022632
OBJ: 1339.16323748
OBJ: 1338.92314042
OBJ: 1337.49895166
OBJ: 1335.32412578
OBJ: 1334.9954297
OBJ: 1334.6988906
OBJ: 1332.97892914


OBJ: 1320.89255527
OBJ: 1320.89213571
OBJ: 1320.88192528
OBJ: 1320.86482873
OBJ: 1320.86293309
OBJ: 1320.85814296
OBJ: 1320.85314478
OBJ: 1320.85265751
OBJ: 1320.85061605
OBJ: 1320.84877754
OBJ: 1320.84855476
OBJ: 1320.84670271
OBJ: 1320.84635123
OBJ: 1320.84378348
OBJ: 1320.84192686
OBJ: 1320.84010123
OBJ: 1320.83622384
OBJ: 1320.83600326
OBJ: 1320.83482965
OBJ: 1320.83450603
OBJ: 1320.83328264
OBJ: 1320.83303178
OBJ: 1320.83240912
OBJ: 1320.83148382
OBJ: 1320.83131408
OBJ: 1320.83044056
OBJ: 1320.83021657
OBJ: 1320.83016011
OBJ: 1320.83000941
OBJ: 1320.82968101
OBJ: 1320.82964885
OBJ: 1320.82952953
OBJ: 1320.82948082
OBJ: 1320.82913917
OBJ: 1320.82911549
OBJ: 1320.82883886
OBJ: 1320.82876349
OBJ: 1320.82849266
OBJ: 1320.82847404
OBJ: 1320.82778886
OBJ: 1320.82741506
OBJ: 1320.82734702
OBJ: 1320.82651824
OBJ: 1320.82547729
OBJ: 1320.82511163
OBJ: 1320.82424565
OBJ: 1320.82385096
OBJ: 1320.82376801
OBJ: 1320.82365407
OBJ: 1320.82345268
OBJ: 1320.82336215
OBJ: 1320.82278978
OBJ: 1320.82

/Users/maryam/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:222: UserWarning: Max. number of function evaluations reached


ZeroDivisionError: integer division or modulo by zero